# table info
`we have Day , Time , halls that are slots for courses : D*T*H`

`posiable varibables is  Day , Time , halls , courses : D*T*H*C`

Choice_4_2_9, and it is defined to be a binary variable (able to take only the integers 1 or 0. If Choice_4_2_9 was 1, it would mean the course_id 4 was present in the square situated in day 2, time 9. (If it was 0, it would mean there was not a 4 there)

constrain is :
- professors available day   🟢 done .ex: (ahmed : sat, mon) -> `we can eliminate days for every course before add it to problem vars`
- course must be in one slot 🟢 done
- make يبدا من الاول الاسبوع 🟢 done
- make يبدا من الاول اليوم 🟢 done
- professors can't teach two couses at same time 🟢 done
- department can't take two couses at same time 🟢 done
- add hours to courses 

In [21]:
from pulp import *
import pandas as pd
from IPython.display import display, HTML
from collections import defaultdict

In [24]:

class schedule:
    def __init__(self):
        self.problem = LpProblem("Schedule_Table", LpMinimize)
        self.days = list(range(1, 6))  # Days 1-5
        self.hours = list(range(1, 11))  # Hours 1-10
        self.halls = list(range(101, 106))  # Halls 101-105
        self.courses = list("abcdefghi")
        self.available_days = {
            'a': [1, 2], 'b': [2, 3], 'c': [1, 2, 3], 'd': [1, 3],
            'e': [1, 2], 'f': [2, 3], 'g': [1, 2, 3], 'h': [1, 3], 'i': [2, 3]
        }
        self.professors = {
            "dr.A": ['a', 'b', 'c'], "dr.D.": ['d', 'e', 'f'],
            "dr.X": ['g', 'h'], "dr.Z": ['i']
        }
        self.departments = {
            "IT": ['a', 'b', 'c', 'd'], "Auto": ['e', 'f', 'g', 'h', 'i']
        }
        self.course_hours = {
            'a': 2, 'b': 1, 'c': 3, 'd': 2, 'e': 1,
            'f': 3, 'g': 2, 'h': 1, 'i': 3
        }
        self.choices = None

    def create_variables(self):
        self.choices = LpVariable.dicts(
            "slot",
            (self.days, self.hours, self.halls, self.courses),
            cat="Binary"
        )

    def constrain_start_hours(self):
        for course in self.courses:
            duration = self.course_hours[course]
            max_start = 10 - duration + 1
            for day in self.days:
                for hall in self.halls:
                    for sh in self.hours:
                        if sh > max_start:
                            self.problem += self.choices[day][sh][hall][course] == 0

    def objective_function(self, day_weight=50, hour_weight=50):
        self.problem += lpSum(
            (d * day_weight + sh * hour_weight) * self.choices[d][sh][hall][course]
            for d in self.days
            for sh in self.hours
            for hall in self.halls
            for course in self.courses
        )

    def add_no_overlap_constraints(self):
        for d in self.days:
            for hall in self.halls:
                for current_h in self.hours:
                    overlapping_vars = []
                    for course in self.courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                overlapping_vars.append(self.choices[d][sh][hall][course])
                    self.problem += lpSum(overlapping_vars) <= 1

    def only_slot(self):
        for course in self.courses:
            total = lpSum(
                self.choices[day][sh][hall][course]
                for day in self.days
                for sh in self.hours
                for hall in self.halls
            )
            self.problem += total == 1

    def constrain_days(self):
        for course, days_allowed in self.available_days.items():
            for day in self.days:
                if day not in days_allowed:
                    for sh in self.hours:
                        for hall in self.halls:
                            self.problem += self.choices[day][sh][hall][course] == 0

    def constrain_departments(self):
        for dept, courses in self.departments.items():
            for day in self.days:
                for current_h in self.hours:
                    dept_vars = []
                    for course in courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                for hall in self.halls:
                                    dept_vars.append(self.choices[day][sh][hall][course])
                    self.problem += lpSum(dept_vars) <= 1

    def constrain_pro(self):
        for prof, courses in self.professors.items():
            for day in self.days:
                for current_h in self.hours:
                    prof_vars = []
                    for course in courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                for hall in self.halls:
                                    prof_vars.append(self.choices[day][sh][hall][course])
                    self.problem += lpSum(prof_vars) <= 1

    def solution_to_dataframe(self):
        course_to_professor = {course: prof for prof, courses in self.professors.items() for course in courses}
        schedule_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

        for day in self.days:
            for sh in self.hours:
                for hall in self.halls:
                    for course in self.courses:
                        if self.choices[day][sh][hall][course].value() == 1:
                            duration = self.course_hours[course]
                            prof = course_to_professor[course]
                            for h in range(sh, sh + duration):
                                if h > 10:
                                    continue
                                schedule_dict[day][h][hall].append(f"{course} ({prof})")

        list_of_dfs = []
        for dept, courses in self.departments.items():
            dept_df = pd.DataFrame(index=self.days, columns=self.hours, dtype=str)
            dept_df.index.name = f'Day/Hour - {dept}'
            for day in self.days:
                for hour in self.hours:
                    entries = []
                    for hall in self.halls:
                        if hall in schedule_dict[day][hour]:
                            for entry in schedule_dict[day][hour][hall]:
                                if entry[0] in courses:
                                    entries.append(f"{entry} (Hall {hall})")
                    dept_df.at[day, hour] = "<br>".join(entries) if entries else ""
            list_of_dfs.append(dept_df)
        return list_of_dfs


In [25]:
def main(display_dfs=False):
    table = schedule()
    table.create_variables()
    table.constrain_start_hours()
    table.constrain_days()
    table.only_slot()
    table.add_no_overlap_constraints()
    table.constrain_pro()
    table.constrain_departments()
    table.objective_function(100, 10)

    status = table.problem.solve()
    if status == LpStatusOptimal and display_dfs:
        print("Schedule found:")
        for df in table.solution_to_dataframe():
            display(HTML(df.to_html(escape=False)))
    else:
        print("No valid schedule found.", LpStatus[status])

if __name__ == '__main__':
    main(True)

Schedule found:


,1,2,3,4,5,6,7,8,9,10
Day/Hour - IT,,,,,,,,,,
1,a (dr.A) (Hall 103),a (dr.A) (Hall 103),d (dr.D.) (Hall 105),d (dr.D.) (Hall 105),c (dr.A) (Hall 101),c (dr.A) (Hall 101),c (dr.A) (Hall 101),,,
2,b (dr.A) (Hall 105),,,,,,,,,
3,,,,,,,,,,
4,,,,,,,,,,
5,,,,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - Auto,,,,,,,,,,
1,e (dr.D.) (Hall 102),h (dr.X) (Hall 104),g (dr.X) (Hall 102),g (dr.X) (Hall 102),,,,,,
2,f (dr.D.) (Hall 104),f (dr.D.) (Hall 104),f (dr.D.) (Hall 104),i (dr.Z) (Hall 104),i (dr.Z) (Hall 104),i (dr.Z) (Hall 104),,,,
3,,,,,,,,,,
4,,,,,,,,,,
5,,,,,,,,,,
